In [2]:
import os, re, sys, shutil
import subprocess as sp
import random, string
import numpy as np
from .utils import *

#DEBUG=False

# load package locations from yaml file, watch! global dict
package_locs = load_package_locations()

def pfunc(seq, package='vienna_2', T=37,     # default package is vienna_2
    constraint=None, motif=None, linear=False,
    dangles=True, noncanonical=False, pseudo=False, dna=False, DIRLOC=None,
    bpps=False, param_file=None, coaxial=True, reweight=None,
    return_free_energy = False, beam_size=100, DEBUG=False):
    ''' Compute partition function for RNA sequence.
        Args:
        seq (str): nucleic acid sequence
        T (float): temperature (Celsius)
        constraint (str): structure constraints
        motif (str): argument to vienna motif 
        linear (bool): call LinearPartition to estimate Z in Vienna or Contrafold
        pseudo (bool): nupack only, make prediction with pseudoknots
        dna (bool): nupack only, make prediction for DNA
        dangles (bool): dangles or not, specifiable for vienna, nupack
        coaxial (bool): coaxial stacking or not, specifiable for rnastructure, vfold
        noncanonical(bool): include noncanonical pairs or not (for contrafold, RNAstructure (Cyclefold))
        beam_size (int): beam size option for LinearPartition.
        Possible packages: 
        'vienna_2', 'vienna_1','contrafold_1','contrafold_2','nupack_95','nupack_99','rnasoft_2007','rnasoft_1999','rnastructure','vfold_0','vfold_1'
        
    Returns
        float: free energy
    '''
    # why does it say that it returns the free energy as a float, when the default argument is set return_free_energy = False?
    
    ### to add Threshknot:
    # add arg: threshknot=False
    # will also need to add a warning if someone tries to use threshknot without using the contrafold package and setting linear=True
        # if threshknot:
            # if pkg not in 'contrafold':
                # print('Warning: ThreshKnot only implemented for Contrafold')
            # elif not linear:
                # print('Warning: ThreshKnot only implemented with LinearPartition')
   
    if DEBUG: load_package_locations(DEBUG=True)

    try:    # try tests for errors
        pkg, version = package.lower().split('_')   # split function splits a string into a list
    except:   # except lets you handle any errors
        pkg, version = package.lower(), None

    if not bpps: # if bpps, already printed these warnings
        if not dangles and pkg not in ['vienna', 'nupack']:
            print('Warning: %s does not support dangles options' % pkg)
        if not coaxial and pkg not in ['rnastructure', 'vfold']:
            print('Warning: %s does not support coaxial options' % pkg)
        if linear and pkg not in ['vienna','contrafold','eternafold']:
            print('Warning: LinearPartition only implemented for vienna, contrafold, eternafold.')

    # if pkg=='eternafold' and not 'eternafoldparams' in package_locs.keys():
    #     raise RuntimeError('Error: need to set path to EternaFold params to use eternafold hotkey.')

    if pseudo and pkg !='nupack':
        raise ValueError('pseudo only for use with nupack')

    if pkg=='vienna':
        # using the package vienna gives you the option to use linear partition or not
        if linear:
            Z, tmp_file = pfunc_linearpartition_(seq, package='vienna',bpps=bpps, beam_size=beam_size,
                return_free_energy=return_free_energy, DEBUG=DEBUG)

        else:
            Z, tmp_file = pfunc_vienna_(seq, version=version, T=T, dangles=dangles,
             constraint=constraint, motif=motif, bpps=bpps, param_file=param_file,
             reweight=reweight, return_free_energy=return_free_energy, DEBUG=DEBUG)
     
    elif pkg=='contrafold':
        # contrafold also has the option to use linear partition
        # this is the package and command we will be using
        if linear:
            Z, tmp_file = pfunc_linearpartition_(seq, package='contrafold', bpps=bpps, beam_size=beam_size,
                return_free_energy=return_free_energy, DEBUG=DEBUG)   
            ### to add Threshknot:
            # add arg: threshknot=threshknot
        else:
            Z, tmp_file = pfunc_contrafold_(seq, version=version, T=T, 
                constraint=constraint, bpps=bpps, param_file=param_file, DIRLOC=DIRLOC,
                return_free_energy=return_free_energy, DEBUG=DEBUG)

    elif pkg=='rnastructure':
        Z, tmp_file = pfunc_rnastructure_(seq, version=version, T=T, coaxial=coaxial, 
            constraint=constraint, bpps=bpps, return_free_energy=return_free_energy, DEBUG=DEBUG)

    elif pkg=='rnasoft':
        if constraint is not None:
            print("ERROR: RNAsoft is unable to handle constraints for calculating \
                partition functions, returning unconstrained Z.")

        Z, tmp_file = pfunc_rnasoft_(seq, version=version, T=T, constraint=constraint,
         bpps=bpps,return_free_energy=return_free_energy, DEBUG=DEBUG)

    elif pkg=='nupack':
        Z, tmp_file = pfunc_nupack_(seq, version=version, dangles=dangles, T=T, pseudo=pseudo, dna=dna, constraint=constraint,
            return_free_energy=return_free_energy, DEBUG=DEBUG)

    elif pkg=='vfold':
        Z, tmp_file = pfunc_vfold_(seq, version=version, T=T, coaxial=coaxial, DEBUG=DEBUG)

    elif pkg=='eternafold':
        if linear:
            Z, tmp_file = pfunc_linearpartition_(seq, package='eternafold', bpps=bpps, beam_size=beam_size,
                return_free_energy=return_free_energy, DEBUG=DEBUG)
        else:
            # Using contrafold code and eternafold params
            if 'eternafoldparams' in package_locs.keys() and 'eternafold' not in package_locs.keys():
                Z, tmp_file = pfunc_contrafold_(seq, T=T, constraint=constraint, 
                    bpps=bpps, param_file=package_locs['eternafoldparams'], DIRLOC=DIRLOC, return_free_energy=return_free_energy, DEBUG=DEBUG)

            elif 'eternafold' in package_locs.keys():
                #Using eternafold code and params in eternafold codebase
                efold_param_file = package_locs['eternafold']+'/../parameters/EternaFoldParams.v1'
                if not os.path.exists(efold_param_file):
                    RuntimeError('Error: Parameters not found at %s' % efold_param_file)

                else:
                    Z, tmp_file = pfunc_contrafold_(seq, T=T, constraint=constraint, 
                        bpps=bpps, param_file=efold_param_file, DIRLOC= package_locs['eternafold'], return_free_energy=return_free_energy, DEBUG=DEBUG)



    else:
        raise ValueError('package %s not understood.' % package)

    if bpps:
        return Z, tmp_file

    else:
        if tmp_file:
            if os.path.exists(tmp_file):
                os.remove(tmp_file)
        return Z

ImportError: attempted relative import with no known parent package

In [ ]:
def pfunc_linearpartition_(seq, bpps=False, package='contrafold', beam_size=100, return_free_energy=False, DEBUG=False):
    LOC = package_locs['linearpartition']
    tmp_file = filename()
    tmp_command = filename()

    if bpps:
        pf_only = 0
    else:
        pf_only = 1

    # args: beamsize, is_sharpturn, is_verbose, bpp_file, bpp_prefix, pf_only, bpp_cutoff,
    #forest_file, mea, gamma, TK, threshold, ThreshKnot_prefix, MEA_prefix, MEA_bpseq

    command=['echo %s | %s/linearpartition_%s' % (seq, LOC, package[0].lower()), str(beam_size),
     '0', '0', tmp_file, '_', str(pf_only), '0.000001', '_', '_', '_','_','_','_','_','_']
                                                                      #TK #0.3
    ### to add Threshknot: 
        # add arg threshknot=False
        # if thresknot:
            # TK = -T
        # else:
            # TK = None
        # put the variable in the command so that if threshknot is specified, -T will be added to the command
            # command: '%s' % (TK)
        # probably do not need to specify threshold since it is default 0.3
        
    with open('%s.sh' % tmp_command,'w') as f:
        f.write(' '.join(command))

    if DEBUG: print(' '.join(command))

    meta_command = ['chmod +x %s.sh; %s.sh' % (tmp_command, tmp_command)]
    p = sp.Popen(meta_command, stdout=sp.PIPE, stderr=sp.PIPE,shell=True)

    stdout, stderr = p.communicate(input=str.encode(seq))

    if DEBUG:
        print('stdout')
        print(stdout)
        print('stderr')
        print(stderr)

    if p.returncode:
        raise Exception('LinearPartition failed: on %s\n%s' % (seq, stderr))

    os.remove("%s.sh" % tmp_command)
    # Note: the linearfold exec says this is free energy in kcal/mol.
    # this is still just cfold log Z

    # linearfold returns two different things depending on which package

    if bpps:
        return 0, tmp_file
    else:

        if package in ['contrafold','eternafold']:
            logZ=float(stderr.decode('utf-8').split(' ')[-1])
            os.remove('_') # tmp file written by linearpartition forest
            if return_free_energy:
                return -1*logZ, None
            else:
                return np.exp(logZ), None

        elif package=='vienna':
            free_energy = float(stderr.decode('utf-8').split(' ')[-2])
            T=37
            os.remove('_') # tmp file written by linearpartition forest
            if return_free_energy:
                return free_energy, None
            else:
                return np.exp(-1*free_energy/(.0019899*(273+T))), None